In [2]:
import mido
from mido import MidiFile, MidiTrack, Message
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Activation, Dropout, Flatten
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [3]:
mid = MidiFile('data/HiroshiYoshimura_Green.mid') 

In [4]:
notes = []
for msg in mid:
    if not msg.is_meta and msg.channel == 0 and msg.type == 'note_on':
        data = msg.bytes()
        notes.append(data[1])

In [5]:
scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(np.array(notes).reshape(-1,1))
notes = list(scaler.transform(np.array(notes).reshape(-1,1)))

In [6]:
# LSTM layers requires that data must have a certain shape
# create list of lists fist
notes = [list(note) for note in notes]

# subsample data for training and prediction
X = []
y = []
# number of notes in a batch
n_prev = 80
for i in range(len(notes)-n_prev):
    X.append(notes[i:i+n_prev])
    y.append(notes[i+n_prev])
    
# save a seed to do prediction later
from sklearn.model_selection import train_test_split
X, X_test, y, y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [7]:
model = Sequential()
model.add(LSTM(256, input_shape=(n_prev, 1), return_sequences=True))
model.add(Dropout(0.6))
model.add(LSTM(128, input_shape=(n_prev, 1), return_sequences=True))
model.add(Dropout(0.6))
model.add(LSTM(64, input_shape=(n_prev, 1), return_sequences=False))
model.add(Dropout(0.6))
model.add(Dense(1))
model.add(Activation('linear'))
optimizer = Adam(lr=0.001)
model.compile(loss='mse', optimizer=optimizer)
filepath="./checkpoint/checkpoint_model_{epoch:02d}.hdf5"
model_save_callback = ModelCheckpoint(filepath, monitor='val_acc', 
                                      verbose=1, save_best_only=False, 
                                      mode='auto', save_freq=50)

In [8]:
model.fit(np.array(X), np.array(y), 50, 10, verbose=1, callbacks=[model_save_callback])

Epoch 1/10
   49/30799 [..............................] - ETA: 2:14:23 - loss: 0.0497
Epoch 00001: saving model to ./checkpoint/checkpoint_model_01.hdf5
   99/30799 [..............................] - ETA: 2:28:16 - loss: 0.0440
Epoch 00001: saving model to ./checkpoint/checkpoint_model_01.hdf5
  149/30799 [..............................] - ETA: 2:33:43 - loss: 0.0424
Epoch 00001: saving model to ./checkpoint/checkpoint_model_01.hdf5
  199/30799 [..............................] - ETA: 2:40:24 - loss: 0.0418
Epoch 00001: saving model to ./checkpoint/checkpoint_model_01.hdf5
  249/30799 [..............................] - ETA: 2:40:02 - loss: 0.0416
Epoch 00001: saving model to ./checkpoint/checkpoint_model_01.hdf5
  299/30799 [..............................] - ETA: 2:38:56 - loss: 0.0410
Epoch 00001: saving model to ./checkpoint/checkpoint_model_01.hdf5
  349/30799 [..............................] - ETA: 2:38:33 - loss: 0.0405
Epoch 00001: saving model to ./checkpoint/checkpoint_model_01.

 2949/30799 [=>............................] - ETA: 2:25:33 - loss: 0.0374
Epoch 00001: saving model to ./checkpoint/checkpoint_model_01.hdf5
 2999/30799 [=>............................] - ETA: 2:24:47 - loss: 0.0374
Epoch 00001: saving model to ./checkpoint/checkpoint_model_01.hdf5
 3049/30799 [=>............................] - ETA: 2:24:31 - loss: 0.0373
Epoch 00001: saving model to ./checkpoint/checkpoint_model_01.hdf5
 3099/30799 [==>...........................] - ETA: 2:24:01 - loss: 0.0375
Epoch 00001: saving model to ./checkpoint/checkpoint_model_01.hdf5
 3149/30799 [==>...........................] - ETA: 2:23:52 - loss: 0.0376
Epoch 00001: saving model to ./checkpoint/checkpoint_model_01.hdf5
 3199/30799 [==>...........................] - ETA: 2:23:20 - loss: 0.0377
Epoch 00001: saving model to ./checkpoint/checkpoint_model_01.hdf5
 3249/30799 [==>...........................] - ETA: 2:23:21 - loss: 0.0378
Epoch 00001: saving model to ./checkpoint/checkpoint_model_01.hdf5
 3299/

 5849/30799 [====>.........................] - ETA: 2:08:46 - loss: 0.0408
Epoch 00001: saving model to ./checkpoint/checkpoint_model_01.hdf5
 5899/30799 [====>.........................] - ETA: 2:08:21 - loss: 0.0409
Epoch 00001: saving model to ./checkpoint/checkpoint_model_01.hdf5
 5949/30799 [====>.........................] - ETA: 2:08:07 - loss: 0.0409
Epoch 00001: saving model to ./checkpoint/checkpoint_model_01.hdf5
 5999/30799 [====>.........................] - ETA: 2:07:48 - loss: 0.0409
Epoch 00001: saving model to ./checkpoint/checkpoint_model_01.hdf5
 6049/30799 [====>.........................] - ETA: 2:07:37 - loss: 0.0410
Epoch 00001: saving model to ./checkpoint/checkpoint_model_01.hdf5
 6099/30799 [====>.........................] - ETA: 2:07:07 - loss: 0.0410
Epoch 00001: saving model to ./checkpoint/checkpoint_model_01.hdf5
 6149/30799 [====>.........................] - ETA: 2:06:37 - loss: 0.0410
Epoch 00001: saving model to ./checkpoint/checkpoint_model_01.hdf5
 6199/

 8749/30799 [=======>......................] - ETA: 1:47:16 - loss: 0.0422
Epoch 00001: saving model to ./checkpoint/checkpoint_model_01.hdf5
 8799/30799 [=======>......................] - ETA: 1:46:55 - loss: 0.0422
Epoch 00001: saving model to ./checkpoint/checkpoint_model_01.hdf5
 8849/30799 [=======>......................] - ETA: 1:46:32 - loss: 0.0422
Epoch 00001: saving model to ./checkpoint/checkpoint_model_01.hdf5
 8899/30799 [=======>......................] - ETA: 1:46:15 - loss: 0.0422
Epoch 00001: saving model to ./checkpoint/checkpoint_model_01.hdf5
 8949/30799 [=======>......................] - ETA: 1:45:59 - loss: 0.0423
Epoch 00001: saving model to ./checkpoint/checkpoint_model_01.hdf5
 8999/30799 [=======>......................] - ETA: 1:45:44 - loss: 0.0423
Epoch 00001: saving model to ./checkpoint/checkpoint_model_01.hdf5
 9049/30799 [=======>......................] - ETA: 1:45:24 - loss: 0.0423
Epoch 00001: saving model to ./checkpoint/checkpoint_model_01.hdf5
 9099/

11649/30799 [==========>...................] - ETA: 1:29:19 - loss: 0.0429
Epoch 00001: saving model to ./checkpoint/checkpoint_model_01.hdf5
11699/30799 [==========>...................] - ETA: 1:29:00 - loss: 0.0429
Epoch 00001: saving model to ./checkpoint/checkpoint_model_01.hdf5
11749/30799 [==========>...................] - ETA: 1:28:42 - loss: 0.0429
Epoch 00001: saving model to ./checkpoint/checkpoint_model_01.hdf5
11799/30799 [==========>...................] - ETA: 1:28:24 - loss: 0.0429
Epoch 00001: saving model to ./checkpoint/checkpoint_model_01.hdf5
11849/30799 [==========>...................] - ETA: 1:28:06 - loss: 0.0429
Epoch 00001: saving model to ./checkpoint/checkpoint_model_01.hdf5
11899/30799 [==========>...................] - ETA: 1:27:48 - loss: 0.0429
Epoch 00001: saving model to ./checkpoint/checkpoint_model_01.hdf5
11949/30799 [==========>...................] - ETA: 1:27:30 - loss: 0.0429
Epoch 00001: saving model to ./checkpoint/checkpoint_model_01.hdf5
11999/

...

17458/30799 [================>.............] - ETA: 58:23 - loss: 0.0447
Epoch 00010: saving model to ./checkpoint/checkpoint_model_10.hdf5
17508/30799 [================>.............] - ETA: 58:08 - loss: 0.0447
Epoch 00010: saving model to ./checkpoint/checkpoint_model_10.hdf5
17558/30799 [================>.............] - ETA: 57:54 - loss: 0.0447
Epoch 00010: saving model to ./checkpoint/checkpoint_model_10.hdf5
17608/30799 [================>.............] - ETA: 57:39 - loss: 0.0447
Epoch 00010: saving model to ./checkpoint/checkpoint_model_10.hdf5
17658/30799 [================>.............] - ETA: 57:25 - loss: 0.0447
Epoch 00010: saving model to ./checkpoint/checkpoint_model_10.hdf5
17708/30799 [================>.............] - ETA: 57:11 - loss: 0.0447
Epoch 00010: saving model to ./checkpoint/checkpoint_model_10.hdf5
17758/30799 [================>.............] - ETA: 56:56 - loss: 0.0447
Epoch 00010: saving model to ./checkpoint/checkpoint_model_10.hdf5
17808/30799 [=======

20408/30799 [==================>...........] - ETA: 44:51 - loss: 0.0447
Epoch 00010: saving model to ./checkpoint/checkpoint_model_10.hdf5
20458/30799 [==================>...........] - ETA: 44:37 - loss: 0.0447
Epoch 00010: saving model to ./checkpoint/checkpoint_model_10.hdf5
20508/30799 [==================>...........] - ETA: 44:23 - loss: 0.0447
Epoch 00010: saving model to ./checkpoint/checkpoint_model_10.hdf5
20558/30799 [===================>..........] - ETA: 44:09 - loss: 0.0447
Epoch 00010: saving model to ./checkpoint/checkpoint_model_10.hdf5
20608/30799 [===================>..........] - ETA: 43:55 - loss: 0.0447
Epoch 00010: saving model to ./checkpoint/checkpoint_model_10.hdf5
20658/30799 [===================>..........] - ETA: 43:42 - loss: 0.0447
Epoch 00010: saving model to ./checkpoint/checkpoint_model_10.hdf5
20708/30799 [===================>..........] - ETA: 43:28 - loss: 0.0447
Epoch 00010: saving model to ./checkpoint/checkpoint_model_10.hdf5
20758/30799 [=======

23358/30799 [=====================>........] - ETA: 31:28 - loss: 0.0448
Epoch 00010: saving model to ./checkpoint/checkpoint_model_10.hdf5
23408/30799 [=====================>........] - ETA: 31:15 - loss: 0.0448
Epoch 00010: saving model to ./checkpoint/checkpoint_model_10.hdf5
23458/30799 [=====================>........] - ETA: 31:02 - loss: 0.0448
Epoch 00010: saving model to ./checkpoint/checkpoint_model_10.hdf5
23508/30799 [=====================>........] - ETA: 30:49 - loss: 0.0448
Epoch 00010: saving model to ./checkpoint/checkpoint_model_10.hdf5
23558/30799 [=====================>........] - ETA: 30:35 - loss: 0.0448
Epoch 00010: saving model to ./checkpoint/checkpoint_model_10.hdf5
23608/30799 [=====================>........] - ETA: 30:22 - loss: 0.0448
Epoch 00010: saving model to ./checkpoint/checkpoint_model_10.hdf5
23658/30799 [======================>.......] - ETA: 30:09 - loss: 0.0448
Epoch 00010: saving model to ./checkpoint/checkpoint_model_10.hdf5
23708/30799 [=======

26308/30799 [========================>.....] - ETA: 19:01 - loss: 0.0448
Epoch 00010: saving model to ./checkpoint/checkpoint_model_10.hdf5
26358/30799 [========================>.....] - ETA: 18:48 - loss: 0.0448
Epoch 00010: saving model to ./checkpoint/checkpoint_model_10.hdf5
26408/30799 [========================>.....] - ETA: 18:35 - loss: 0.0448
Epoch 00010: saving model to ./checkpoint/checkpoint_model_10.hdf5
26458/30799 [========================>.....] - ETA: 18:22 - loss: 0.0448
Epoch 00010: saving model to ./checkpoint/checkpoint_model_10.hdf5
26508/30799 [========================>.....] - ETA: 18:09 - loss: 0.0448
Epoch 00010: saving model to ./checkpoint/checkpoint_model_10.hdf5
26558/30799 [========================>.....] - ETA: 17:56 - loss: 0.0448
Epoch 00010: saving model to ./checkpoint/checkpoint_model_10.hdf5
26608/30799 [========================>.....] - ETA: 17:43 - loss: 0.0448
Epoch 00010: saving model to ./checkpoint/checkpoint_model_10.hdf5
26658/30799 [=======

29258/30799 [===========================>..] - ETA: 6:26 - loss: 0.0447
Epoch 00010: saving model to ./checkpoint/checkpoint_model_10.hdf5
29308/30799 [===========================>..] - ETA: 6:13 - loss: 0.0447
Epoch 00010: saving model to ./checkpoint/checkpoint_model_10.hdf5
29358/30799 [===========================>..] - ETA: 6:01 - loss: 0.0447
Epoch 00010: saving model to ./checkpoint/checkpoint_model_10.hdf5
29408/30799 [===========================>..] - ETA: 5:48 - loss: 0.0447
Epoch 00010: saving model to ./checkpoint/checkpoint_model_10.hdf5
29458/30799 [===========================>..] - ETA: 5:35 - loss: 0.0447
Epoch 00010: saving model to ./checkpoint/checkpoint_model_10.hdf5
29508/30799 [===========================>..] - ETA: 5:23 - loss: 0.0447
Epoch 00010: saving model to ./checkpoint/checkpoint_model_10.hdf5
29558/30799 [===========================>..] - ETA: 5:10 - loss: 0.0447
Epoch 00010: saving model to ./checkpoint/checkpoint_model_10.hdf5
29608/30799 [==============

In [16]:
prediction = model.predict(np.array(X_test))
prediction = np.squeeze(prediction)
prediction = np.squeeze(scaler.inverse_transform(prediction.reshape(-1,1)))
prediction = [int(i) for i in prediction]

In [17]:
track = MidiTrack()
t = 0
for note in prediction:
    # 147 means note_on
    # 67 is velosity
    note = np.asarray([147, note, 70])
    bytes = note.astype(int)
    msg = Message.from_bytes(bytes[0:3])
    t += 1
    msg.time = t
    track.append(msg)
mid.tracks.append(track)
mid.save('green4.mid')

In [13]:
model.save('green_model.h5')